In [15]:
import os
%pwd  # this tell us which path we are currently working , so based on the below output path we are working under the research file
os.chdir("C:\datascience End to End Projects\End-to-End-sentimental-analysis-with-NLP")  #  but i would like to work with main ProjectML_with_MLFlow file , so for getting i step back in path inorder to enter the main project file i used this command os.chdir("../")

In [22]:
%pwd

'C:\\datascience End to End Projects\\End-to-End-sentimental-analysis-with-NLP'

In [23]:
# This is called the entity 
from dataclasses import dataclass # here i imported the dataclass from the dataclasses
from pathlib import Path  # here i imported path from pathlib

# here entity means DataIngestionConfig which it returns all the variables like root_dir,source_URL  and etc 
@dataclass(frozen=True) # here i declared the dataclass decorator
class DataIngestionConfig:  # here i have created a class and named as DataIngestionConfig ,and it is not a python class because we need to declare the self to the variables if it is a python class, it is data class  and whenever i define the configuration fucntion , this class should my return function , the below are the varaible it do return 
    root_dir: Path    # these are variable which i have declared inside the class 
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [24]:
from NLP_PROJECT.constants import * # here iam importing everthing which is present in the constants->__init__.py file into inside the data_ingestion.ipynb
from NLP_PROJECT.utils.common import read_yaml, create_directories # here iam importing the read_yaml, create_directories which are presenting inside the utils,common files into PROJECTML in which the file is data_ingestion.ipynb

In [25]:
class ConfigurationManager:  # here iam creating class called ConfigurationManager
    def __init__( # inisde this class iam reading all the yaml files which iam calling it from constants->__init__.py file and iam mentioning inside the class varaiable 
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath) # and here iam giving read_yaml path here and iam giving the path after that then it will return all the configuration in the variable
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

# now i will create artifacts root in the side of the vscode project one of the path and the below i will define the data ingestion cofiguration function
    # the above one  entity which inside 4 variables needs to return by this below fucntion
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir]) # here iam creating the root directory, and iam reading the config from the configurationManager class and iam going to access all the data ingestion from the config.yaml file 

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,  # that how iam accessing all the things like root_dir,source_url and etc from config.yaml file and finally this fucntion do return all this variables 
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [26]:
import os
import zipfile # here iam importing the zipfile to unzip the chest images data
import gdown  # here iam importing the gdown to download the chest image data from google drive
from NLP_PROJECT import logger 
from NLP_PROJECT.utils.common import get_size  # here iam importing the get_size to get to know the size of the data

In [27]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try:  # do follow the explaination of trais.ipynb file for to know how to download the data from googledrive
            dataset_url = self.config.source_URL  # here iam getting the dataset url 
            zip_download_dir = self.config.local_data_file  # here iam saving the path of the images data of google drive 
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]  # 
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):  # here by using the extract_zip_file fucntion we are  going to unzip the images dataset 
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [29]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-11 17:43:14,299: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-11 17:43:14,312: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-11 17:43:14,315: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-11 17:43:14,320: INFO: common: created directory at: artifacts]
[2024-04-11 17:43:14,322: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-11 17:43:14,326: INFO: 519299881: Downloading data from https://drive.google.com/file/d/1rydvEK4UJlutIBvevtYB_Jdpkw4XvfaB/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1rydvEK4UJlutIBvevtYB_Jdpkw4XvfaB
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1rydvEK4UJlutIBvevtYB_Jdpkw4XvfaB&confirm=t&uuid=ad52f0ae-6562-4679-afc6-b561ff7c7ed3
To: C:\datascience End to End Projects\End-to-End-sentimental-analysis-with-NLP\artifacts\data_ingestion\data.zip
100%|██████████| 27.0M/27.0M [00:03<00:00, 8.32MB/s]

[2024-04-11 17:43:19,449: INFO: 519299881: Downloaded data from https://drive.google.com/file/d/1rydvEK4UJlutIBvevtYB_Jdpkw4XvfaB/view?usp=sharing into file artifacts/data_ingestion/data.zip]
